In [ ]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras.models import load_model
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



In [ ]:
df3 = pd.read_csv("run_KnuckleFinger_LSTM_Jan_20190125_111527-tag-acc.csv", delimiter=",")
df4 = pd.read_csv("run_KnuckleFinger_LSTM_Jan_20190125_111527-tag-val_acc.csv", delimiter=",")

In [ ]:
plt.plot(df3.Step, df3.Value, label='training')
plt.plot(df4.Step, df4.Value, label='validation')
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.ylim(0.4,1)
plt.legend()
plt.savefig('pres.png', dpi =1000)

In [ ]:
df3.head()

In [ ]:
df1 = pd.read_csv("run_KnuckleFinger_Jan_20190109_103953-tag-acc.csv", delimiter=",")
df2 = pd.read_csv("run_KnuckleFinger_Jan_20190109_103953-tag-val_acc.csv", delimiter=",")

In [ ]:
df1.head()

In [ ]:
plt.plot(df1.Step, df1.Value, label='training')
plt.plot(df2.Step, df2.Value, label='validation')
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.legend()
plt.savefig('pres.png', dpi =1000)

In [ ]:
df = pd.read_csv('accel_a.csv', delimiter=';')

In [ ]:
cv1 = pd.read_csv('CrossValid/results_20190127_154838.csv', delimiter=';')
cv2 = pd.read_csv('CrossValid/results_20190130_120228.csv', delimiter=';')
cv_list = [cv1,cv2]

In [ ]:
cv = pd.concat(cv_list)

In [ ]:
cv.head()

In [ ]:
from ast import literal_eval
cv["ValAcc"] = cv["ValAcc"].apply(literal_eval)
cv["Acc"] = cv["Acc"].apply(literal_eval)

In [ ]:
cv["ValAccLast"] = cv["ValAcc"].apply(lambda x: x[-1])
cv["AccLast"] = cv["Acc"].apply(lambda x: x[-1])

In [ ]:
cv_new = cv[["TestIds","ValAccLast", "AccLast"]].copy()

In [ ]:
cv_new["TestIds"] = cv_new["TestIds"].apply(literal_eval)
cv_new["TestID"] = cv_new["TestIds"].apply(lambda x: x[0])
cv_new["Acc"] = cv_new["AccLast"].round(4)
cv_new["ValAcc"] = cv_new["ValAccLast"].round(4)
cv_new2 = cv_new[["TestID","ValAcc", "Acc"]].copy()

In [ ]:
cv_new2.sort_values("TestID").to_csv("CrossValidResults.csv")

In [ ]:
cv["ValAccLast"].mean()

In [ ]:
cv["AccLast"].mean()

In [ ]:
cv["ValAccLast"].std()

In [ ]:
cv["AccLast"].std()

In [ ]:
model = load_model("ModelSnapshots/KnuckleFinger_20181223_173148.h5", compile=False)

In [ ]:
acc_list = []
valAcc_list = []
for i in range(len(cv)):
    acc_list.append(cv["Acc"].iloc[i])
    valAcc_list.append(cv["ValAcc"].iloc[i])
acc_means = np.mean(acc_list, axis=0)
valAcc_means = np.mean(valAcc_list, axis = 0)

In [ ]:
steps = range(len(cv["Acc"].iloc[0]))

In [ ]:
plt.plot(steps, acc_means, label='training')
plt.plot(steps, valAcc_means, label='validation')
plt.xlabel('Epochs', fontsize=15)
plt.ylabel('Accuracy', fontsize=15)
plt.legend()
plt.savefig('pres.jpg')

In [ ]:
from keras.utils import plot_model
import pydot
plot_model(model,to_file='demo.png',show_shapes=True, show_layer_names=False, rankdir='TB')

In [ ]:
dfSurvey = pd.read_csv("SurveyResults.csv")
dfSurvey = dfSurvey.drop(["Timestamp", "Gender", "Age"], axis=1)
dfSurvey = dfSurvey.drop(dfSurvey.columns[-1], axis=1)

In [ ]:
SurveyMeans = dfSurvey.mean()

In [ ]:
labels = ["Tap", "Two Tap", "Swipe left/right/up/down", "Swipe up with two", "Swipe down with two", "Swipe left/right with two",
         "Circle", "Circle with two", "Arrowhead left/right", "Checkmark", "Г", "L and mirrored L", "S", "Press and rotate"]
plt.barh([i for i in range(14)], SurveyMeans)
plt.yticks([i for i in range(14)], labels)
plt.savefig("SurveyResults.pdf", bbox_inches='tight', dpi=300)